In [ ]:
# Install colab_ssh on google colab
!pip install colab_ssh --upgrade
!pip install dm-acme[jax,tensorflow]
!pip install dm-acme[envs]
!pip install dm-sonnet
!pip install dm-launchpad
!pip install trfl
!pip install pyyaml==5.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 314 kB 5.2 MB/s 
     |████████████████████████████████| 3.8 MB 34.5 MB/s 
     |████████████████████████████████| 72 kB 562 kB/s 
     |████████████████████████████████| 342 kB 42.5 MB/s 
     |████████████████████████████████| 197 kB 51.7 MB/s 
     |████████████████████████████████| 145 kB 46.2 MB/s 
     |████████████████████████████████| 117 kB 49.2 MB/s 
     |████████████████████████████████| 6.4 MB 33.8 MB/s 
     |████████████████████████████████| 4.0 MB 24.0 MB/s 
     |████████████████████████████████| 668.3 MB 18 kB/s 
     |████████████████████████████████| 5.7 MB 28.4 MB/s 
     |████████████████████████████████| 462 kB 34.9 MB/s 
     |████████████████████████████████| 217 kB 33.8 MB/s 
     |████████████████████████████████| 596 kB 35.9

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.express as px

from plotly.subplots import make_subplots
import plotly.figure_factory as ff

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


# **Section 1: Strategy P&L Analysis**

## **General Functions**

In [ ]:
def generate_stat(folder, eval_quantiles, fig, agent_name):
  eval_df = pd.read_csv(folder)
  eval_loop_df = eval_df[['episode','step_pnl']].groupby('episode').aggregate('sum')
  eval_loop_df.columns = ['episode_return']
  gamma_exposure_df = eval_df[['episode','gamma_before_hedge', 'gamma_after_hedge']]
  gamma_exposure_df['gamma_hedged_amount'] = (gamma_exposure_df['gamma_before_hedge']-gamma_exposure_df['gamma_after_hedge'])*np.sign(gamma_exposure_df['gamma_before_hedge'])
  gamma_exposure_df['gamma_before_hedge_level'] = gamma_exposure_df['gamma_before_hedge'].abs()
  gamma_exposure_df = gamma_exposure_df.groupby('episode').aggregate('sum')
  gamma_exposure = gamma_exposure_df['gamma_hedged_amount']/gamma_exposure_df['gamma_before_hedge_level']

  vega_exposure_df = eval_df[['episode','vega_before_hedge', 'vega_after_hedge']]
  vega_exposure_df['vega_hedged_amount'] = (vega_exposure_df['vega_before_hedge']-vega_exposure_df['vega_after_hedge'])*np.sign(vega_exposure_df['vega_before_hedge'])
  vega_exposure_df['vega_before_hedge_level'] = vega_exposure_df['vega_before_hedge'].abs()
  vega_exposure_df = vega_exposure_df.groupby('episode').aggregate('sum')
  vega_exposure = vega_exposure_df['vega_hedged_amount']/vega_exposure_df['vega_before_hedge_level']

  stats_str = '{0: <20}'.format(agent_name) + ' pnl mean: {:.2f} | std: {:.6f} | Mean-SD: {:.2f} | gamma-ratio: {:5.2f}({:5.2f}/{:5.2f}) | vega-ratio: {:5.2f}({:5.2f}/{:5.2f}) '.format(
          eval_loop_df['episode_return'].mean(),
          eval_loop_df['episode_return'].std(),
          eval_loop_df['episode_return'].mean()-1.645*eval_loop_df['episode_return'].std(),
          gamma_exposure.mean(), gamma_exposure.quantile(0.10), gamma_exposure.quantile(0.90),
          vega_exposure.mean(), vega_exposure.quantile(0.10), vega_exposure.quantile(0.90))
  for q in eval_quantiles:
    stats_str += '| var{:.0f}: {:8.2f} '.format(
              100*q, eval_loop_df['episode_return'].quantile(1-q))
  for q in eval_quantiles:
    stats_str += '| cvar{:.0f}: {:8.2f} '.format(
              100*q, eval_loop_df.loc[eval_loop_df['episode_return']<=eval_loop_df['episode_return'].quantile(1-q),'episode_return'].mean())
      
  print(stats_str)
  fig.add_trace(go.Histogram(
      x=eval_loop_df.episode_return.values,
      histnorm='percent', name=f'{agent_name} Terminal P&L', opacity=0.75,
      nbinsx=150
  ))
  return eval_loop_df.episode_return.values

## **Section 4 Table 1 (GBM)**

**0.5% Transaction Cost**

In [ ]:
workfolder = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table1/TC05/RL/mean_std/spread=0.005_obj=meanstd_threshold=0.95_critic=qr_v=0.0_hedttm=30'
workfolder2 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table1/TC05/RL/var/spread=0.005_obj=var_threshold=0.95_critic=qr_v=0.0_hedttm=30'
workfolder3 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table1/TC05/RL/cvar/spread=0.005_obj=cvar_threshold=0.95_critic=qr_v=0.0_hedttm=30'
gkfolder = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table1/TC05/Baseline/delta/greekhedge_spread=0.005_v=0.0_hedttm=30'
gkfolder2 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table1/TC05/Baseline/gamma/greekhedge_spread=0.005_v=0.0_hedttm=30'

In [ ]:
eval_quantiles = [0.95, 0.85, 0.75,0.65]
fig = go.Figure()
generate_stat(
       f'{workfolder}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'Mean-SD')
generate_stat(
       f'{workfolder2}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'VaR95')
generate_stat(
       f'{workfolder3}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'CVaR95')

generate_stat(
       f'{gkfolder}/logs/eval_delta_env/logs.csv', 
       eval_quantiles, fig, 'Delta')
generate_stat( 
       f'{gkfolder2}/logs/eval_gamma_env/logs.csv', 
       eval_quantiles, fig, 'Delta-Gamma')

Mean-SD              pnl mean: -3.00 | std: 1.481112 | Mean-SD: -5.44 | gamma-ratio:  0.83( 0.77/ 0.89) | vega-ratio:  0.14( 0.04/ 0.28) | var95:    -5.46 | var85:    -4.31 | var75:    -3.74 | var65:    -3.35 | cvar95:    -6.71 | cvar85:    -5.45 | cvar75:    -4.87 | cvar65:    -4.49 
VaR95                pnl mean: -2.80 | std: 1.654244 | Mean-SD: -5.52 | gamma-ratio:  0.75( 0.67/ 0.83) | vega-ratio:  0.14( 0.04/ 0.27) | var95:    -5.47 | var85:    -4.32 | var75:    -3.69 | var65:    -3.27 | cvar95:    -6.66 | cvar85:    -5.42 | cvar75:    -4.84 | cvar65:    -4.45 
CVaR95               pnl mean: -2.92 | std: 1.617529 | Mean-SD: -5.58 | gamma-ratio:  0.79( 0.69/ 0.88) | vega-ratio:  0.14( 0.04/ 0.28) | var95:    -5.55 | var85:    -4.33 | var75:    -3.79 | var65:    -3.35 | cvar95:    -6.78 | cvar85:    -5.48 | cvar75:    -4.90 | cvar65:    -4.52 
Delta                pnl mean: 0.05 | std: 14.987090 | Mean-SD: -24.61 | gamma-ratio:  0.00( 0.00/ 0.00) | vega-ratio:  0.00( 0.00/ 0.00) | va

array([-2.04677753, -2.59370248, -3.29789809, ..., -3.16958477,
       -3.29017394, -2.22347131])

**1% Transaction Cost**

In [ ]:
workfolder = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table1/TC10/RL/mean_std/spread=0.01_obj=meanstd_threshold=0.95_critic=qr_v=0.0_hedttm=30'
workfolder2 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table1/TC10/RL/var/spread=0.01_obj=var_threshold=0.95_critic=qr_v=0.0_hedttm=30'
workfolder3 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table1/TC10/RL/cvar/spread=0.01_obj=cvar_threshold=0.95_critic=qr_v=0.0_hedttm=30'
gkfolder = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table1/TC10/Baseline/delta/greekhedge_spread=0.01_v=0.0_hedttm=30'
gkfolder2 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table1/TC10/Baseline/gamma/greekhedge_spread=0.01_v=0.0_hedttm=30'

In [ ]:
eval_quantiles = [0.95, 0.85, 0.75,0.65]
fig = go.Figure()
generate_stat(
       f'{workfolder}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'Mean-SD')
rl_pnl = generate_stat(
       f'{workfolder2}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'VaR95')
generate_stat(
       f'{workfolder3}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'CVaR95')

delta_pnl = generate_stat(
       f'{gkfolder}/logs/eval_delta_env/logs.csv', 
       eval_quantiles, fig, 'Delta')
gamma_pnl = generate_stat( 
       f'{gkfolder2}/logs/eval_gamma_env/logs.csv', 
       eval_quantiles, fig, 'Delta-Gamma')

Mean-SD              pnl mean: -4.58 | std: 2.292844 | Mean-SD: -8.36 | gamma-ratio:  0.57( 0.47/ 0.67) | vega-ratio:  0.12( 0.04/ 0.22) | var95:    -8.57 | var85:    -6.80 | var75:    -5.95 | var65:    -5.27 | cvar95:    -9.97 | cvar85:    -8.35 | cvar75:    -7.55 | cvar65:    -6.99 
VaR95                pnl mean: -4.51 | std: 2.404798 | Mean-SD: -8.47 | gamma-ratio:  0.56( 0.46/ 0.67) | vega-ratio:  0.11( 0.04/ 0.21) | var95:    -8.63 | var85:    -6.88 | var75:    -5.96 | var65:    -5.24 | cvar95:   -10.03 | cvar85:    -8.43 | cvar75:    -7.62 | cvar65:    -7.03 
CVaR95               pnl mean: -4.71 | std: 2.306072 | Mean-SD: -8.51 | gamma-ratio:  0.60( 0.51/ 0.68) | vega-ratio:  0.12( 0.04/ 0.22) | var95:    -8.59 | var85:    -6.95 | var75:    -6.01 | var65:    -5.43 | cvar95:   -10.02 | cvar85:    -8.44 | cvar75:    -7.64 | cvar65:    -7.09 
Delta                pnl mean: 0.05 | std: 14.987090 | Mean-SD: -24.61 | gamma-ratio:  0.00( 0.00/ 0.00) | vega-ratio:  0.00( 0.00/ 0.00) | va

In [ ]:
hist_data = [delta_pnl,gamma_pnl,rl_pnl]
group_labels = ['Delta-Neutral','Delta-Gamma-Neutral','RL']
fig = ff.create_distplot(hist_data, group_labels, show_hist=False)
fig.update_layout(
  title_text='Terminal P&L Distribution (Spread=1%)', # title of plot
)
fig.show()

**Transaction Cost 2%**

In [ ]:
workfolder = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table1/TC20/RL/mean_std/spread=0.02_obj=meanstd_threshold=0.95_critic=qr_v=0.0_hedttm=30'
workfolder2 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table1/TC20/RL/var/spread=0.02_obj=var_threshold=0.95_critic=qr_v=0.0_hedttm=30'
workfolder3 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table1/TC20/RL/cvar/spread=0.02_obj=cvar_threshold=0.95_critic=qr_v=0.0_hedttm=30'
gkfolder = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table1/TC20/Baseline/delta/greekhedge_spread=0.02_v=0.0_hedttm=30'
gkfolder2 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table1/TC20/Baseline/gamma/greekhedge_spread=0.02_v=0.0_hedttm=30'

In [ ]:
eval_quantiles = [0.95, 0.85, 0.75,0.65]
fig = go.Figure()
rl_pnl = generate_stat(
       f'{workfolder}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'Mean-SD')
generate_stat(
       f'{workfolder2}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'VaR95')
generate_stat(
       f'{workfolder3}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'CVaR95')

generate_stat(
       f'{gkfolder}/logs/eval_delta_env/logs.csv', 
       eval_quantiles, fig, 'Delta')
generate_stat( 
       f'{gkfolder2}/logs/eval_gamma_env/logs.csv', 
       eval_quantiles, fig, 'Delta-Gamma')

Mean-SD              pnl mean: -5.91 | std: 4.147970 | Mean-SD: -12.73 | gamma-ratio:  0.30( 0.20/ 0.41) | vega-ratio:  0.08( 0.04/ 0.13) | var95:   -12.98 | var85:   -10.02 | var75:    -8.44 | var65:    -7.24 | cvar95:   -15.54 | cvar85:   -12.71 | cvar75:   -11.30 | cvar65:   -10.30 
VaR95                pnl mean: -5.15 | std: 4.543931 | Mean-SD: -12.62 | gamma-ratio:  0.24( 0.16/ 0.33) | vega-ratio:  0.07( 0.04/ 0.11) | var95:   -13.05 | var85:    -9.51 | var75:    -7.87 | var65:    -6.52 | cvar95:   -16.05 | cvar85:   -12.69 | cvar75:   -11.08 | cvar65:    -9.96 
CVaR95               pnl mean: -5.78 | std: 4.059144 | Mean-SD: -12.46 | gamma-ratio:  0.29( 0.22/ 0.36) | vega-ratio:  0.08( 0.04/ 0.12) | var95:   -12.86 | var85:    -9.80 | var75:    -8.16 | var65:    -6.99 | cvar95:   -15.37 | cvar85:   -12.47 | cvar75:   -11.04 | cvar65:   -10.05 
Delta                pnl mean: 0.05 | std: 14.987090 | Mean-SD: -24.61 | gamma-ratio:  0.00( 0.00/ 0.00) | vega-ratio:  0.00( 0.00/ 0.00) |

array([-12.11795693, -12.61868473, -12.63293532, ..., -12.01200063,
       -13.70497912, -10.68597609])

## **Section 4 Table 2 (SABR)**

**0.5% Transaction Cost**

In [ ]:
workfolder = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC05/RL/mean_std/spread=0.005_obj=meanstd_threshold=0.95_critic=qr_v=0.3_hedttm=30'
workfolder2 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC05/RL/var/spread=0.005_obj=var_threshold=0.95_critic=qr_v=0.3_hedttm=30'
workfolder3 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC05/RL/cvar/spread=0.005_obj=cvar_threshold=0.95_critic=qr_v=0.3_hedttm=30'
gkfolder = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC05/Baseline/delta/greekhedge_spread=0.005_v=0.3_hedttm=30'
gkfolder2 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC05/Baseline/gamma/greekhedge_spread=0.005_v=0.3_hedttm=30'
gkfolder3 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC05/Baseline/vega/greekhedge_spread=0.005_v=0.3_hedttm=30'

workfolder4 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC05/RL/mean_std/spread=0.005_obj=meanstd_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder5 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC05/RL/var/spread=0.005_obj=var_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder6 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC05/RL/cvar/spread=0.005_obj=cvar_threshold=0.95_critic=qr_v=0.3_hedttm=90'
gkfolder4 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC05/Baseline/delta/greekhedge_spread=0.005_v=0.3_hedttm=90'
gkfolder5 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC05/Baseline/gamma/greekhedge_spread=0.005_v=0.3_hedttm=90'
gkfolder6 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC05/Baseline/vega/greekhedge_spread=0.005_v=0.3_hedttm=90'

In [ ]:
eval_quantiles = [0.95, 0.85, 0.75,0.65]
fig = go.Figure()
print("30 Days Result")
rl_pnl = generate_stat(
       f'{workfolder}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'Mean-SD')
generate_stat(
       f'{workfolder2}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'VaR95')
generate_stat(
       f'{workfolder3}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'CVaR95')
generate_stat(
       f'{gkfolder}/logs/eval_delta_env/logs.csv', 
       eval_quantiles, fig, 'Delta')
generate_stat( 
       f'{gkfolder2}/logs/eval_gamma_env/logs.csv', 
       eval_quantiles, fig, 'Delta-Gamma')
generate_stat( 
       f'{gkfolder3}/logs/eval_vega_env/logs.csv', 
       eval_quantiles, fig, 'Delta-Vega')
print("90 Days Result")
rl_pnl = generate_stat(
       f'{workfolder4}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'Mean-SD')
generate_stat(
       f'{workfolder5}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'VaR95')
generate_stat(
       f'{workfolder6}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'CVaR95')
generate_stat(
       f'{gkfolder4}/logs/eval_delta_env/logs.csv', 
       eval_quantiles, fig, 'Delta')
generate_stat( 
       f'{gkfolder5}/logs/eval_gamma_env/logs.csv', 
       eval_quantiles, fig, 'Delta-Gamma')
generate_stat( 
       f'{gkfolder6}/logs/eval_vega_env/logs.csv', 
       eval_quantiles, fig, 'Delta-Vega')

30 Days Result
Mean-SD              pnl mean: -2.75 | std: 9.123019 | Mean-SD: -17.76 | gamma-ratio:  0.53( 0.24/ 0.73) | vega-ratio:  0.17( 0.06/ 0.31) | var95:   -17.67 | var85:   -10.61 | var75:    -7.43 | var65:    -5.29 | cvar95:   -24.52 | cvar85:   -17.16 | cvar75:   -13.82 | cvar65:   -11.68 
VaR95                pnl mean: -3.25 | std: 9.970477 | Mean-SD: -19.65 | gamma-ratio:  0.61( 0.41/ 0.81) | vega-ratio:  0.18( 0.05/ 0.36) | var95:   -19.31 | var85:   -11.37 | var75:    -8.38 | var65:    -6.04 | cvar95:   -27.49 | cvar85:   -18.85 | cvar75:   -15.20 | cvar65:   -12.89 
CVaR95               pnl mean: -3.44 | std: 9.342895 | Mean-SD: -18.81 | gamma-ratio:  0.73( 0.48/ 0.98) | vega-ratio:  0.16( 0.04/ 0.30) | var95:   -19.19 | var85:   -11.49 | var75:    -8.32 | var65:    -6.07 | cvar95:   -26.06 | cvar85:   -18.37 | cvar75:   -14.92 | cvar65:   -12.69 
Delta                pnl mean: 0.22 | std: 21.871468 | Mean-SD: -35.76 | gamma-ratio:  0.00( 0.00/ 0.00) | vega-ratio: -0.00

array([ -7.61137308, -10.63379189,  -5.8192754 , ...,  -7.21480022,
        -5.37170242,  -3.84990464])

**1% Transaction Cost**

In [ ]:
workfolder = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC10/RL/mean_std/spread=0.01_obj=meanstd_threshold=0.95_critic=qr_v=0.3_hedttm=30'
workfolder2 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC10/RL/var/spread=0.01_obj=var_threshold=0.95_critic=qr_v=0.3_hedttm=30'
workfolder3 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC10/RL/cvar/spread=0.01_obj=cvar_threshold=0.95_critic=qr_v=0.3_hedttm=30'
gkfolder = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC10/Baseline/delta/greekhedge_spread=0.01_v=0.3_hedttm=30'
gkfolder2 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC10/Baseline/gamma/greekhedge_spread=0.01_v=0.3_hedttm=30'
gkfolder3 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC10/Baseline/vega/greekhedge_spread=0.01_v=0.3_hedttm=30'

workfolder4 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC10/RL/mean_std/spread=0.01_obj=meanstd_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder5 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC10/RL/var/spread=0.01_obj=var_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder6 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC10/RL/cvar/spread=0.01_obj=cvar_threshold=0.95_critic=qr_v=0.3_hedttm=90'
gkfolder4 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC10/Baseline/delta/greekhedge_spread=0.01_v=0.3_hedttm=90'
gkfolder5 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC10/Baseline/gamma/greekhedge_spread=0.01_v=0.3_hedttm=90'
gkfolder6 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC10/Baseline/vega/greekhedge_spread=0.01_v=0.3_hedttm=90'

In [ ]:
eval_quantiles = [0.95, 0.85, 0.75,0.65]
fig = go.Figure()
print("30 Days Result")
rl_pnl = generate_stat(
       f'{workfolder}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'Mean-SD')
generate_stat(
       f'{workfolder2}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'VaR95')
generate_stat(
       f'{workfolder3}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'CVaR95')
generate_stat(
       f'{gkfolder}/logs/eval_delta_env/logs.csv', 
       eval_quantiles, fig, 'Delta')
generate_stat( 
       f'{gkfolder2}/logs/eval_gamma_env/logs.csv', 
       eval_quantiles, fig, 'Delta-Gamma')
generate_stat( 
       f'{gkfolder3}/logs/eval_vega_env/logs.csv', 
       eval_quantiles, fig, 'Delta-Vega')
print("90 Days Result")
rl_pnl = generate_stat(
       f'{workfolder4}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'Mean-SD')
generate_stat(
       f'{workfolder5}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'VaR95')
generate_stat(
       f'{workfolder6}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'CVaR95')
generate_stat(
       f'{gkfolder4}/logs/eval_delta_env/logs.csv', 
       eval_quantiles, fig, 'Delta')
generate_stat( 
       f'{gkfolder5}/logs/eval_gamma_env/logs.csv', 
       eval_quantiles, fig, 'Delta-Gamma')
generate_stat( 
       f'{gkfolder6}/logs/eval_vega_env/logs.csv', 
       eval_quantiles, fig, 'Delta-Vega')

30 Days Result
Mean-SD              pnl mean: -4.40 | std: 9.502285 | Mean-SD: -20.03 | gamma-ratio:  0.50( 0.38/ 0.62) | vega-ratio:  0.13( 0.04/ 0.23) | var95:   -19.94 | var85:   -12.51 | var75:    -9.46 | var65:    -7.05 | cvar95:   -27.14 | cvar85:   -19.43 | cvar75:   -16.00 | cvar65:   -13.78 
VaR95                pnl mean: -3.98 | std: 10.085474 | Mean-SD: -20.57 | gamma-ratio:  0.45( 0.31/ 0.57) | vega-ratio:  0.12( 0.04/ 0.20) | var95:   -20.22 | var85:   -12.73 | var75:    -9.10 | var65:    -6.82 | cvar95:   -27.98 | cvar85:   -19.89 | cvar75:   -16.21 | cvar65:   -13.85 
CVaR95               pnl mean: -4.00 | std: 9.747739 | Mean-SD: -20.04 | gamma-ratio:  0.42( 0.29/ 0.56) | vega-ratio:  0.10( 0.04/ 0.18) | var95:   -20.26 | var85:   -12.75 | var75:    -9.40 | var65:    -6.85 | cvar95:   -26.81 | cvar85:   -19.65 | cvar75:   -16.18 | cvar65:   -13.86 
Delta                pnl mean: 0.22 | std: 21.871468 | Mean-SD: -35.76 | gamma-ratio:  0.00( 0.00/ 0.00) | vega-ratio: -0.0

array([-14.42323398, -16.58393544, -11.42405453, ..., -13.71335726,
       -11.02408419,  -9.83422746])

**Transaction Cost 2%**

In [ ]:
workfolder = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC20/RL/mean_std/spread=0.02_obj=meanstd_threshold=0.95_critic=qr_v=0.3_hedttm=30'
workfolder2 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC20/RL/var/spread=0.02_obj=var_threshold=0.95_critic=qr_v=0.3_hedttm=30'
workfolder3 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC20/RL/cvar/spread=0.02_obj=cvar_threshold=0.95_critic=qr_v=0.3_hedttm=30'
gkfolder = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC20/Baseline/delta/greekhedge_spread=0.02_v=0.3_hedttm=30'
gkfolder2 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC20/Baseline/gamma/greekhedge_spread=0.02_v=0.3_hedttm=30'
gkfolder3 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC20/Baseline/vega/greekhedge_spread=0.02_v=0.3_hedttm=30'

workfolder4 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC20/RL/mean_std/spread=0.02_obj=meanstd_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder5 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC20/RL/var/spread=0.02_obj=var_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder6 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC20/RL/cvar/spread=0.02_obj=cvar_threshold=0.95_critic=qr_v=0.3_hedttm=90'
gkfolder4 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC20/Baseline/delta/greekhedge_spread=0.02_v=0.3_hedttm=90'
gkfolder5 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC20/Baseline/gamma/greekhedge_spread=0.02_v=0.3_hedttm=90'
gkfolder6 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC20/Baseline/vega/greekhedge_spread=0.02_v=0.3_hedttm=90'

In [ ]:
eval_quantiles = [0.95, 0.85, 0.75,0.65]
fig = go.Figure()
print("30 Days Result")
rl_pnl = generate_stat(
       f'{workfolder}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'Mean-SD')
generate_stat(
       f'{workfolder2}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'VaR95')
generate_stat(
       f'{workfolder3}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'CVaR95')
generate_stat(
       f'{gkfolder}/logs/eval_delta_env/logs.csv', 
       eval_quantiles, fig, 'Delta')
generate_stat( 
       f'{gkfolder2}/logs/eval_gamma_env/logs.csv', 
       eval_quantiles, fig, 'Delta-Gamma')
generate_stat( 
       f'{gkfolder3}/logs/eval_vega_env/logs.csv', 
       eval_quantiles, fig, 'Delta-Vega')
print("90 Days Result")
rl_pnl = generate_stat(
       f'{workfolder4}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'Mean-SD')
generate_stat(
       f'{workfolder5}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'VaR95')
generate_stat(
       f'{workfolder6}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'CVaR95')
generate_stat(
       f'{gkfolder4}/logs/eval_delta_env/logs.csv', 
       eval_quantiles, fig, 'Delta')
generate_stat( 
       f'{gkfolder5}/logs/eval_gamma_env/logs.csv', 
       eval_quantiles, fig, 'Delta-Gamma')
generate_stat( 
       f'{gkfolder6}/logs/eval_vega_env/logs.csv', 
       eval_quantiles, fig, 'Delta-Vega')

30 Days Result
Mean-SD              pnl mean: -7.01 | std: 10.430630 | Mean-SD: -24.17 | gamma-ratio:  0.33( 0.21/ 0.47) | vega-ratio:  0.11( 0.04/ 0.19) | var95:   -24.08 | var85:   -16.57 | var75:   -12.70 | var65:    -9.79 | cvar95:   -31.95 | cvar85:   -23.85 | cvar75:   -20.09 | cvar65:   -17.54 
VaR95                pnl mean: -5.70 | std: 10.893821 | Mean-SD: -23.62 | gamma-ratio:  0.29( 0.17/ 0.41) | vega-ratio:  0.07( 0.03/ 0.12) | var95:   -23.85 | var85:   -15.65 | var75:   -11.65 | var65:    -8.78 | cvar95:   -31.65 | cvar85:   -23.30 | cvar75:   -19.41 | cvar65:   -16.76 
CVaR95               pnl mean: -5.62 | std: 11.046302 | Mean-SD: -23.79 | gamma-ratio:  0.27( 0.15/ 0.38) | vega-ratio:  0.07( 0.03/ 0.12) | var95:   -23.71 | var85:   -15.77 | var75:   -11.73 | var65:    -8.73 | cvar95:   -31.57 | cvar85:   -23.31 | cvar75:   -19.44 | cvar65:   -16.80 
Delta                pnl mean: 0.22 | std: 21.871468 | Mean-SD: -35.76 | gamma-ratio:  0.00( 0.00/ 0.00) | vega-ratio: -0

array([-28.04695577, -28.48422253, -22.63361279, ..., -26.71047135,
       -22.32884774, -21.80287309])

# **Section 5 Robustness Test Table 1**

In [ ]:
workfolder = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Robustness_Table_1/TC10/RL/0.0_mean_std_eval/spread=0.01_obj=meanstd_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder2 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Robustness_Table_1/TC10/RL/0.0_var_eval/spread=0.01_obj=var_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder3 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Robustness_Table_1/TC10/RL/0.0_cvar_eval/spread=0.01_obj=cvar_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder4 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC10/RL/mean_std/spread=0.01_obj=meanstd_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder5 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC10/RL/var/spread=0.01_obj=var_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder6 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC10/RL/cvar/spread=0.01_obj=cvar_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder7 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Robustness_Table_1/TC10/RL/0.6_mean_std_eval/spread=0.01_obj=meanstd_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder8 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Robustness_Table_1/TC10/RL/0.6_var_eval/spread=0.01_obj=var_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder9 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Robustness_Table_1/TC10/RL/0.6_cvar_eval/spread=0.01_obj=cvar_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder10 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Robustness_Table_1/TC10/RL/0.15_mean_std_eval/spread=0.01_obj=meanstd_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder11 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Robustness_Table_1/TC10/RL/0.15_var_eval/spread=0.01_obj=var_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder12 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Robustness_Table_1/TC10/RL/0.15_cvar_eval/spread=0.01_obj=cvar_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder13 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Robustness_Table_1/TC10/RL/0.45_mean_std_eval/spread=0.01_obj=meanstd_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder14 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Robustness_Table_1/TC10/RL/0.45_var_eval/spread=0.01_obj=var_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder15 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Robustness_Table_1/TC10/RL/0.45_cvar_eval/spread=0.01_obj=cvar_threshold=0.95_critic=qr_v=0.3_hedttm=90'

In [ ]:
eval_quantiles = [0.95, 0.85, 0.75,0.65]
fig = go.Figure()
rl_pnl = generate_stat(
       f'{workfolder}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'Mean-SD - 0.0')
generate_stat(
       f'{workfolder2}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'VaR95 - 0.0')
generate_stat(
       f'{workfolder3}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'CVaR95 - 0.0')
generate_stat(
       f'{workfolder4}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'Mean-SD - 0.3')
generate_stat(
       f'{workfolder5}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'VaR95 - 0.3')
generate_stat(
       f'{workfolder6}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'CVaR95 - 0.3')
generate_stat(
       f'{workfolder7}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'Mean-SD - 0.6')
generate_stat(
       f'{workfolder8}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'VaR95 - 0.6')
generate_stat(
       f'{workfolder9}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'CVaR95 - 0.6')
generate_stat(
       f'{workfolder10}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'Mean-SD - 0.15')
generate_stat(
       f'{workfolder11}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'VaR95 - 0.15')
generate_stat(
       f'{workfolder12}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'CVaR95 - 0.15')
generate_stat(
       f'{workfolder13}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'Mean-SD - 0.45')
generate_stat(
       f'{workfolder14}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'VaR95 - 0.45')
generate_stat(
       f'{workfolder15}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'CVaR95 - 0.45')

Mean-SD - 0.0        pnl mean: -6.03 | std: 10.491876 | Mean-SD: -23.29 | gamma-ratio:  0.17( 0.09/ 0.26) | vega-ratio:  0.16( 0.07/ 0.26) | var95:   -24.38 | var85:   -16.03 | var75:   -11.84 | var65:    -8.90 | cvar95:   -31.04 | cvar85:   -23.31 | cvar75:   -19.52 | cvar65:   -16.89 
VaR95 - 0.0          pnl mean: -6.43 | std: 10.207762 | Mean-SD: -23.22 | gamma-ratio:  0.18( 0.12/ 0.26) | vega-ratio:  0.16( 0.06/ 0.26) | var95:   -23.96 | var85:   -16.07 | var75:   -12.04 | var65:    -9.29 | cvar95:   -30.71 | cvar85:   -23.09 | cvar75:   -19.40 | cvar65:   -16.88 
CVaR95 - 0.0         pnl mean: -7.08 | std: 10.810706 | Mean-SD: -24.86 | gamma-ratio:  0.21( 0.13/ 0.31) | vega-ratio:  0.17( 0.04/ 0.29) | var95:   -25.75 | var85:   -17.32 | var75:   -13.08 | var65:   -10.11 | cvar95:   -32.79 | cvar85:   -24.75 | cvar75:   -20.83 | cvar65:   -18.15 
Mean-SD - 0.3        pnl mean: -6.21 | std: 7.540365 | Mean-SD: -18.61 | gamma-ratio:  0.14( 0.06/ 0.23) | vega-ratio:  0.37( 0.28/ 0.46

array([ -6.10660623,  -3.70040197, -13.72185003, ..., -16.67600445,
        -2.92126885,   3.5872744 ])

# **Section 5 Robustness Test Table 2**

In [ ]:
workfolder = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Robustness_Table_2/TC10/RL/mean_std/init_vol_0.2_eval/spread=0.01_obj=meanstd_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder2 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Robustness_Table_2/TC10/RL/var/init_vol_0.2_eval/spread=0.01_obj=var_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder3 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Robustness_Table_2/TC10/RL/cvar/init_vol_0.2_eval/spread=0.01_obj=cvar_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder4 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC10/RL/mean_std/spread=0.01_obj=meanstd_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder5 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC10/RL/var/spread=0.01_obj=var_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder6 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Table2/TC10/RL/cvar/spread=0.01_obj=cvar_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder7 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Robustness_Table_2/TC10/RL/mean_std/init_vol_0.4_eval/spread=0.01_obj=meanstd_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder8 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Robustness_Table_2/TC10/RL/var/init_vol_0.4_eval/spread=0.01_obj=var_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder9 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Robustness_Table_2/TC10/RL/cvar/init_vol_0.4_eval/spread=0.01_obj=cvar_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder10 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Robustness_Table_2/TC10/RL/mean_std/init_vol_0.25_eval/spread=0.01_obj=meanstd_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder11 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Robustness_Table_2/TC10/RL/var/init_vol_0.25_eval/spread=0.01_obj=var_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder12 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Robustness_Table_2/TC10/RL/cvar/init_vol_0.25_eval/spread=0.01_obj=cvar_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder13 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Robustness_Table_2/TC10/RL/mean_std/init_vol_0.35_eval/spread=0.01_obj=meanstd_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder14 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Robustness_Table_2/TC10/RL/var/init_vol_0.35_eval/spread=0.01_obj=var_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder15 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Robustness_Table_2/TC10/RL/cvar/init_vol_0.35_eval/spread=0.01_obj=cvar_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder16 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Robustness_Table_2/TC10/RL/mean_std/init_vol_0.1_eval/spread=0.01_obj=meanstd_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder17 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Robustness_Table_2/TC10/RL/var/init_vol_0.1_eval/spread=0.01_obj=var_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder18 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Robustness_Table_2/TC10/RL/cvar/init_vol_0.1_eval/spread=0.01_obj=cvar_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder19 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Robustness_Table_2/TC10/RL/mean_std/init_vol_0.5_eval/spread=0.01_obj=meanstd_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder20 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Robustness_Table_2/TC10/RL/var/init_vol_0.5_eval/spread=0.01_obj=var_threshold=0.95_critic=qr_v=0.3_hedttm=90'
workfolder21 = '/content/gdrive/MyDrive/FinHub/Gamma and Vega Hedging Using Deep Distributional Reinforcement Learning/gammahedging_paper_codebase/logs/batch14/Robustness_Table_2/TC10/RL/cvar/init_vol_0.5_eval/spread=0.01_obj=cvar_threshold=0.95_critic=qr_v=0.3_hedttm=90'

In [ ]:
eval_quantiles = [0.95, 0.85, 0.75,0.65]
fig = go.Figure()
rl_pnl = generate_stat(
       f'{workfolder}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'Mean-SD - 0.2')
generate_stat(
       f'{workfolder2}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'VaR95 - 0.2')
generate_stat(
       f'{workfolder3}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'CVaR95 - 0.2')
generate_stat(
       f'{workfolder4}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'Mean-SD - 0.3')
generate_stat(
       f'{workfolder5}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'VaR95 - 0.3')
generate_stat(
       f'{workfolder6}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'CVaR95 - 0.3')
generate_stat(
       f'{workfolder7}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'Mean-SD - 0.4')
generate_stat(
       f'{workfolder8}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'VaR95 - 0.4')
generate_stat(
       f'{workfolder9}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'CVaR95 - 0.4')
generate_stat(
       f'{workfolder10}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'Mean-SD - 0.25')
generate_stat(
       f'{workfolder11}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'VaR95 - 0.25')
generate_stat(
       f'{workfolder12}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'CVaR95 - 0.25')
generate_stat(
       f'{workfolder13}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'Mean-SD - 0.35')
generate_stat(
       f'{workfolder14}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'VaR95 - 0.35')
generate_stat(
       f'{workfolder15}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'CVaR95 - 0.35')
generate_stat(
       f'{workfolder16}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'Mean-SD - 0.1')
generate_stat(
       f'{workfolder17}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'VaR95 - 0.1')
generate_stat(
       f'{workfolder18}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'CVaR95 - 0.1')
generate_stat(
       f'{workfolder19}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'Mean-SD - 0.5')
generate_stat(
       f'{workfolder20}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'VaR95 - 0.5')
generate_stat(
       f'{workfolder21}/logs/eval_env/logs.csv',
       eval_quantiles, fig, 'CVaR95 - 0.5')

Mean-SD - 0.2        pnl mean: -8.94 | std: 7.621251 | Mean-SD: -21.48 | gamma-ratio:  0.21( 0.06/ 0.39) | vega-ratio:  0.50( 0.35/ 0.66) | var95:   -22.05 | var85:   -16.22 | var75:   -13.22 | var65:   -11.07 | cvar95:   -27.31 | cvar85:   -21.56 | cvar75:   -18.79 | cvar65:   -16.89 
VaR95 - 0.2          pnl mean: -8.37 | std: 7.298185 | Mean-SD: -20.37 | gamma-ratio:  0.22( 0.07/ 0.39) | vega-ratio:  0.53( 0.35/ 0.68) | var95:   -19.61 | var85:   -14.73 | var75:   -12.37 | var65:   -10.59 | cvar95:   -24.65 | cvar85:   -19.38 | cvar75:   -17.01 | cvar65:   -15.42 
CVaR95 - 0.2         pnl mean: -7.79 | std: 8.042086 | Mean-SD: -21.02 | gamma-ratio:  0.18( 0.05/ 0.33) | vega-ratio:  0.46( 0.26/ 0.67) | var95:   -21.27 | var85:   -14.88 | var75:   -11.95 | var65:   -10.02 | cvar95:   -27.07 | cvar85:   -20.65 | cvar75:   -17.72 | cvar65:   -15.77 
Mean-SD - 0.3        pnl mean: -6.21 | std: 7.540365 | Mean-SD: -18.61 | gamma-ratio:  0.14( 0.06/ 0.23) | vega-ratio:  0.37( 0.28/ 0.46) |

array([-10.05420826,  -3.81040234, -11.49048567, ..., -15.18543347,
        -8.45828324,   1.96351702])